In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
import warnings
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric

# 0 Importing Data

In [17]:
# Load your dataset
merge_df_scaled = pd.read_csv('../raw_data/cleaned_merge_df_top10.csv')
merge_df_scaled['date'] = pd.to_datetime(merge_df_scaled['date'])
merge_df_scaled.set_index('date', inplace=True)

merge_df_scaled.head()
# 382600 rows × 64 columns

,id,item_id,dept_id,cat_id,store_id,state_id,sales,weekday,wday,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
date,,,,,,,,,,,,,,,,,
2011-01-29,FOODS_2_197_CA_1_validation,FOODS_2_197,FOODS_2,FOODS,CA_1,CA,38,Saturday,1,0,0,0,0,0,0,0,2.98
2011-01-29,FOODS_3_080_CA_1_validation,FOODS_3_080,FOODS_3,FOODS,CA_1,CA,33,Saturday,1,0,0,0,0,0,0,0,1.48
2011-01-29,FOODS_3_090_CA_1_validation,FOODS_3_090,FOODS_3,FOODS,CA_1,CA,107,Saturday,1,0,0,0,0,0,0,0,1.25
2011-01-29,FOODS_3_120_CA_1_validation,FOODS_3_120,FOODS_3,FOODS,CA_1,CA,0,Saturday,1,0,0,0,0,0,0,0,0.00
2011-01-29,FOODS_3_252_CA_1_validation,FOODS_3_252,FOODS_3,FOODS,CA_1,CA,19,Saturday,1,0,0,0,0,0,0,0,1.48


# 1. Defining Model Functions

In [33]:
def perform_prophet(product_data):

    product_data.reset_index(inplace=True,names="date")
    
    prophet_product_df = product_data[["id","date","sales"]]
    prophet_product_df.columns = ["id","ds","y"]
    prophet_product_df['ds'] = pd.to_datetime(prophet_df['ds'])
    
    data_train = prophet_product_df.iloc[:-28]
    data_test = prophet_product_df.iloc[-28:]
    X_train = data_train["ds"]
    y_train = data_train["y"]
    X_test = data_test["ds"]
    y_test = data_test["y"]
    
    fbp = Prophet()

    model = fbp.fit(data_train)
    
    predict_placeholder = fbp.make_future_dataframe(28,freq="D")
    
    # Predict on the test data
    y_pred = fbp.predict(predict_placeholder[-28:])
    

    # Calculate and return the error metric for the current fold
    mae = mean_absolute_error(y_test, y_pred["yhat"])
    
    return model, mae

In [34]:
def perform_auto_arima(product_data):
    data_train = product_data.iloc[:-28]
    data_test = product_data.iloc[-28:]
    y_train = data_train["sales"]
    y_test = data_test["sales"]

    # Fit ARIMA model on the training data using auto_arima to find the best (p, d, q)
    model = auto_arima(y_train, start_p=0, start_q=0, max_p=5, max_q=5, d=1,
                       seasonal=True, trace=False, error_action='ignore', 
                       suppress_warnings=True, stepwise=True)
    
    # Predict on the test data
    predictions = model.predict(n_periods=len(y_test))

    # Calculate and return the error metric for the current fold
    mae = mean_absolute_error(y_test, predictions)
    
    return model, mae

# 2.Running all models in a loop to find for each product with lowest score

In [35]:
models_list = ["ARIMA","Prophet"]

In [36]:
from pmdarima import auto_arima

# Dictionary to store MAE results for each unique time-series identified by id
product_results = {}
average_mae = []

# Iterate over each unique product series identified by id
for id in merge_df_scaled['id'].unique()[:10]:
    print(f"Analyzing product: {id}")
    product_data = merge_df_scaled[merge_df_scaled['id'] == id]

    # Results list for the current product time-series
    results = {}
    best_score = 999.99
    best_model_name = ""



    #Looping all models
    for model_name in models_list:

        if model_name == "ARIMA":
            #TODO: Add 5-fold split here for another loop (or inside the model function?) and then take the average score per model as their mae score
            
            # Fit ARIMA model on the training data using auto_arima to find the best (p, d, q)
            model, mae = perform_auto_arima(product_data)
            results[model_name] = {"mae": mae, "model": model}
            if mae < best_score:
                best_score = mae
                best_model = model
                best_model_name = model_name

        #elif model_name == "ExponentialSmoothing":

            # To be built
            #model, mae = perform_exp_smth(product_data)
            #results[model_name] = {"mae": mae, "model": model}
            #if mae < best_score:
             #   best_score = mae
              #  best_model = model
               # best_model_name = model_name

        elif model_name == "Prophet":

            model, mae = perform_prophet(product_data)
            results[model_name] = {"mae": mae, "model": model}
            if mae < best_score:
                best_score = mae
                best_model = model
                best_model_name = model_name


    #Printing results for this product
    print(results)
    print(f"Model results for {id}")
    print(f"Best model: {best_model_name}")
    print(f"Best score: {best_score}")

    average_mae.append(best_score)

    # Store the average MAE for the current product time-series
    product_results[id] = {"best_score": best_score, "best_model": best_model_name, "model": best_model}

    #Store the best model in a pkl file
    filename = f'../models/{id}_model.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(best_model, f)

# Create a DataFrame to store the results
results_df_arima = pd.DataFrame(product_results.items(), columns=['id', 'MAE'])

# Set the 'id' column as the index
results_df_arima.set_index('id', inplace=True)

average_mae = np.mean(average_mae)

print(f"Total average MAE: {average_mae}")


Analyzing product: FOODS_2_197_CA_1_validation


/tmp/ipykernel_70471/1215182328.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prophet_product_df['ds'] = pd.to_datetime(prophet_df['ds'])
14:24:10 - cmdstanpy - INFO - Chain [1] start processing
14:24:10 - cmdstanpy - INFO - Chain [1] done processing


Model results for FOODS_2_197_CA_1_validation
Best model: ARIMA
Best score: 8.726633436944372
Analyzing product: FOODS_3_080_CA_1_validation


/tmp/ipykernel_70471/1215182328.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prophet_product_df['ds'] = pd.to_datetime(prophet_df['ds'])
14:24:21 - cmdstanpy - INFO - Chain [1] start processing
14:24:22 - cmdstanpy - INFO - Chain [1] done processing


Model results for FOODS_3_080_CA_1_validation
Best model: Prophet
Best score: 5.460836328763631
Analyzing product: FOODS_3_090_CA_1_validation


...

Model results for FOODS_3_587_CA_1_validation
Best model: Prophet
Best score: 9.546208647532103
Analyzing product: FOODS_3_714_CA_1_validation


/tmp/ipykernel_70471/1215182328.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prophet_product_df['ds'] = pd.to_datetime(prophet_df['ds'])
14:30:18 - cmdstanpy - INFO - Chain [1] start processing
14:30:18 - cmdstanpy - INFO - Chain [1] done processing


Model results for FOODS_3_714_CA_1_validation
Best model: ARIMA
Best score: 5.518778222112147
Analyzing product: FOODS_3_808_CA_1_validation


/tmp/ipykernel_70471/1215182328.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prophet_product_df['ds'] = pd.to_datetime(prophet_df['ds'])
14:31:13 - cmdstanpy - INFO - Chain [1] start processing
14:31:13 - cmdstanpy - INFO - Chain [1] done processing


Model results for FOODS_3_808_CA_1_validation
Best model: ARIMA
Best score: 3.219758128147768
                                                                           MAE
id                                                                            
FOODS_2_197_CA_1_validation  {'best_score': 8.726633436944372, 'best_model'...
FOODS_3_080_CA_1_validation  {'best_score': 5.460836328763631, 'best_model'...
FOODS_3_090_CA_1_validation  {'best_score': 17.08460121656075, 'best_model'...
FOODS_3_120_CA_1_validation  {'best_score': 24.60149551782473, 'best_model'...
FOODS_3_252_CA_1_validation  {'best_score': 6.680438583319882, 'best_model'...
FOODS_3_555_CA_1_validation  {'best_score': 4.34514653363894, 'best_model':...
FOODS_3_586_CA_1_validation  {'best_score': 7.991362870463669, 'best_model'...
FOODS_3_587_CA_1_validation  {'best_score': 9.546208647532103, 'best_model'...
FOODS_3_714_CA_1_validation  {'best_score': 5.518778222112147, 'best_model'...
FOODS_3_808_CA_1_validation  {'best_s

# ----------- Jonas Cleaned and Optimized until here -------------------

In [37]:
import optuna

# Define the objective function
def objective(trial, y_train, tscv):
    trend = trial.suggest_categorical('trend', ['add'])
    seasonal = trial.suggest_categorical('seasonal', [None, 'add'])
    seasonal_periods = trial.suggest_categorical('seasonal_periods', [None, 4, 7, 12])
    
    product_results = []
    for train_index, test_index in tscv.split(y_train):
        y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]

        # Fit Holt-Winters model on the training data
        model = ExponentialSmoothing(y_train_fold, trend=trend, seasonal=seasonal, seasonal_periods=seasonal_periods,freq='D')
        fitted_model = model.fit(optimized=True)

        # Predict on the test data
        predictions = fitted_model.forecast(steps=len(y_test_fold))

        # Calculate and store the error metric
        mae = mean_absolute_error(y_test_fold, predictions)
        product_results.append(mae)

    # Average MAE for this product
    average_mae = np.mean(product_results)
    return average_mae

# Create a dictionary to store results
results_dict = {}

# Iterate over each unique product ID
for id in merge_df_scaled['id'].unique()[:10]:

    # Create a study object
    study = optuna.create_study(direction='minimize')
    
    print(f"Optimizing hyperparameters for product: {id}")
    product_data = merge_df_scaled[merge_df_scaled['id'] == id]
    y_train = product_data['sales']
    
    # Initialize time series cross-validator
    tscv = TimeSeriesSplit(n_splits=5)
    
    # Run the optimization process for the current product
    study.optimize(lambda trial: objective(trial, y_train, tscv), n_trials=10)

    # Get the best hyperparameters and the corresponding best MAE
    best_params = study.best_params
    best_mae = study.best_value
    
    # Store the results in the dictionary
    results_dict[id] = {'ExpSmoothing_params': best_params, 'ExpSmoothing_MAE': best_mae}

# Print the results dictionary
print(results_dict)



ModuleNotFoundError: No module named 'optuna'

In [8]:
# Convert the results dictionary to a DataFrame
results_df_exp = pd.DataFrame(results_dict).T.reset_index()
results_df_exp.columns = ['id', 'ExpSmoothing_params', 'ExpSmoothing_MAE']

# Set the 'Product ID' column as the index
results_df_exp.set_index('id', inplace=True)
results_df_exp

,ExpSmoothing_params,ExpSmoothing_MAE
id,,
HOBBIES_1_008_CA_1_validation,"{'trend': 'add', 'seasonal': 'add', 'seasonal_...",7.610342
HOBBIES_1_015_CA_1_validation,"{'trend': 'add', 'seasonal': 'add', 'seasonal_...",6.258854
HOBBIES_1_016_CA_1_validation,"{'trend': 'add', 'seasonal': None, 'seasonal_p...",4.346778
HOBBIES_1_019_CA_1_validation,"{'trend': 'add', 'seasonal': 'add', 'seasonal_...",5.017659
HOBBIES_1_032_CA_1_validation,"{'trend': 'add', 'seasonal': None, 'seasonal_p...",5.799883
HOBBIES_1_048_CA_1_validation,"{'trend': 'add', 'seasonal': None, 'seasonal_p...",6.325378
HOBBIES_1_067_CA_1_validation,"{'trend': 'add', 'seasonal': 'add', 'seasonal_...",6.161814
HOBBIES_1_169_CA_1_validation,"{'trend': 'add', 'seasonal': None, 'seasonal_p...",5.085608
HOBBIES_1_178_CA_1_validation,"{'trend': 'add', 'seasonal': 'add', 'seasonal_...",9.991154


In [9]:
# Merge the two DataFrames based on the product ID
comparison_df = pd.merge(results_df_arima, results_df_exp, left_index=True, right_index=True, suffixes=('_arima', '_exp'))
comparison_df['Best MAE'] = comparison_df[['ARIMA_MAE', 'ExpSmoothing_MAE']].min(axis=1)
comparison_df['Best Method'] = comparison_df.apply(lambda row: 'ARIMA' if row['Best MAE'] == row['ARIMA_MAE'] else 'Exponential Smoothing', axis=1)

comparison_df


,ARIMA_MAE,ExpSmoothing_params,ExpSmoothing_MAE,Best MAE,Best Method
id,,,,,
HOBBIES_1_008_CA_1_validation,6.531440,"{'trend': 'add', 'seasonal': 'add', 'seasonal_...",7.610342,6.53144,ARIMA
HOBBIES_1_015_CA_1_validation,5.591126,"{'trend': 'add', 'seasonal': 'add', 'seasonal_...",6.258854,5.591126,ARIMA
HOBBIES_1_016_CA_1_validation,4.480105,"{'trend': 'add', 'seasonal': None, 'seasonal_p...",4.346778,4.346778,Exponential Smoothing
HOBBIES_1_019_CA_1_validation,5.017305,"{'trend': 'add', 'seasonal': 'add', 'seasonal_...",5.017659,5.017305,ARIMA
HOBBIES_1_032_CA_1_validation,5.426358,"{'trend': 'add', 'seasonal': None, 'seasonal_p...",5.799883,5.426358,ARIMA
HOBBIES_1_048_CA_1_validation,6.173675,"{'trend': 'add', 'seasonal': None, 'seasonal_p...",6.325378,6.173675,ARIMA
HOBBIES_1_067_CA_1_validation,5.050134,"{'trend': 'add', 'seasonal': 'add', 'seasonal_...",6.161814,5.050134,ARIMA
HOBBIES_1_169_CA_1_validation,5.745132,"{'trend': 'add', 'seasonal': None, 'seasonal_p...",5.085608,5.085608,Exponential Smoothing
HOBBIES_1_178_CA_1_validation,7.523928,"{'trend': 'add', 'seasonal': 'add', 'seasonal_...",9.991154,7.523928,ARIMA
